In [53]:
%autosave 0
%matplotlib inline

Autosave disabled


In [54]:
import os
import errno
import shutil
from matplotlib import pyplot as plt
import numpy as np
import cv2
import functools
import operator
import random

In [55]:
INPUT_IMAGE_DIR = "/src/input/JPEG_Convert_Resolution_1024"
FACES_IMAGE_DIR = INPUT_IMAGE_DIR + "/faces"
OPENCV_DIR = "/usr/local/share/OpenCV"
OPENCV_FACE_CASCADE = OPENCV_DIR + '/haarcascades/haarcascade_frontalface_default.xml'
OPENCV_EYE_CASCADE = OPENCV_DIR + '/haarcascades/haarcascade_eye.xml'
RESET_FILES_FLAG = True
SHOW_IMAGES_FLAG = False
LIST_FILES_FLAG = False
INPUT_SAMPLE_SIZE = 100

In [56]:
def make_directory(directory, clear=RESET_FILES_FLAG):
    if clear: shutil.rmtree(directory)
    try: os.makedirs(directory)
    except OSError as e:
        if e.errno != errno.EEXIST: raise

def maybe_increment_path(file_path, sep="_"):
    file_path_base, file_ext = os.path.splitext(file_path)
    increment = 0
    while True:
        full_file_path = "%s%s%s%s" % (file_path_base, sep, increment, file_ext)
        if not os.path.exists(full_file_path): return full_file_path
        increment += 1

def show_image(image):
    plt.imshow(img_rgb)
    plt.show()

In [57]:
INPUT_IMAGE_FILES = list(map(lambda x: INPUT_IMAGE_DIR + "/" + x, os.listdir(INPUT_IMAGE_DIR)))
make_directory(FACES_IMAGE_DIR)

In [58]:
#Temporary Variables - for testing
TEST_IMAGE_SHORT = "slwa_b1851997_5_master.jpg"
TEST_IMAGE_FULL = INPUT_IMAGE_DIR + "/" + TEST_IMAGE_SHORT

try: INPUT_IMAGE_FILES_SAMPLE = random.sample(INPUT_IMAGE_FILES, INPUT_SAMPLE_SIZE)
except: pass
if LIST_FILES_FLAG: print(INPUT_IMAGE_FILES_SAMPLE)

In [59]:
def save_face(image_file, face):
    face_file = image_file.replace(INPUT_IMAGE_DIR, FACES_IMAGE_DIR)
    face_file = maybe_increment_path(face_file)
    cv2.imwrite(face_file, face)

def find_faces(image_file, show=SHOW_IMAGES_FLAG):
    img_bgr = cv2.imread(image_file) #Loads into BGR color space by default
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB) #convert img to RGB 
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY) #Converts into grayscale for face detection
    img_gray_eq = cv2.equalizeHist(img_gray) #Normalizes histogram of contrast
    
    face_cascade = cv2.CascadeClassifier(OPENCV_FACE_CASCADE)
    faces = face_cascade.detectMultiScale(img_gray_eq, scaleFactor=1.2, minNeighbors=5, minSize=(30,30))
    for (x,y,w,h) in faces:
        save_face(image_file, img_bgr[y:(y+h), x:(x+w)])
        if show: cv2.rectangle(img_rgb,(x,y),(x+w,y+h),(255,0,0),2)
    
    if show: show_image(img_rgb)

    return faces

faces = [find_faces(image_file) for image_file in INPUT_IMAGE_FILES_SAMPLE]
if LIST_FILES_FLAG: print(os.listdir(FACES_IMAGE_DIR))